In [1]:
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-11.8'
from pytorch_nndct import QatProcessor
import sys
import yaml
import QHydraSalsa
sys.path.append('/ari/users/ibaskaya/projeler/hydrasalsa/utils')

import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from glob import glob

from dskittiwaymo import SegmentationDataset
from lovasz import Lovasz_softmax
from parser1 import Parser
from mappings import kitti, kitti_normalized_frequencies ,kitticolormap,sem2sem, kitti_lm_inv,waymocolormap,waymo,sensor_kitti,sensor_waymo,waymo_lmap, waymo_lmap_inv,waymovalidfreqs
from iou_eval import iouEval
from combinedloader import KittiWaymoTrainLoader


[VAIQ_WARN][QUANTIZER_TORCH_NO_XIR]: Can't find xir package in your environment.

[VAIQ_NOTE]: Loading NNDCT kernels...


In [2]:
num_classes = 23
batch_number = 2
batch_size,workers = 24, 24
max_epochs = 150             
learning_rate = 0.01     
warmup_epochs = 1            
momentum = 0.9              
lr_decay = 0.99                
weight_decay = 0.0001        
batch_size = 2                
epsilon_w = 0.001 

freqsum = sum(waymovalidfreqs)
frequencies = [i/freqsum for i in waymovalidfreqs]

root_waymo = '/ari/users/ibaskaya/projeler/hydrasalsa/data/waymo'
root_kitti = '/ari/users/ibaskaya/projeler/hydrasalsa/data/kittismall'

train_sequences = [0,1,2,3,4,5,6,7,9,10]
valid_sequences = [8]
test_sequences = None
labels_waymo = waymo
color_map_waymo = waymocolormap
learning_map_waymo = waymo_lmap
learning_map_inv_waymo = waymo_lmap_inv

labels_kitti = kitti
color_map_kitti = kitticolormap
learning_map_kitti = sem2sem
learning_map_inv_kitti = kitti_lm_inv


max_points_waymo=170000
max_points_kitti = 150000
gt=True
transform=False
iswaymo = False
istrain = False
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('Using ',device)


Using  cuda


In [3]:
float_model = QHydraSalsa.HydraSalsa(nclasses=[20,23], inchannels=5)
state_dict = torch.load('/ari/users/ibaskaya/projeler/hydrasalsa/best_kitti_state_dict.pth')#, map_location=torch.device('cpu'))
float_model.load_state_dict(state_dict)
float_model.to(device)

HydraSalsa(
  (salsa): Salsa(
    (encoder): SalsaEncoder(
      (downCntx): ResContextBlock(
        (conv1): Conv2d(5, 32, kernel_size=(1, 1), stride=(1, 1))
        (act1): ReLU6()
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act2): ReLU6()
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
        (act3): ReLU6()
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (skip_add): Add()
      )
      (downCntx2): ResContextBlock(
        (conv1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
        (act1): ReLU6()
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act2): ReLU6()
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(32, 32, 

In [4]:
inputs = torch.randn((1,5,64,2048)).to(device)
qat_processor = QatProcessor(float_model, inputs, bitwidth=8)
model = qat_processor.trainable_model(allow_reused_module=True)


[VAIQ_NOTE]: Quant config file is empty, use default quant configuration

[VAIQ_NOTE]: Quantization calibration process start up...

[VAIQ_NOTE]: =>Quant Module is in 'cuda'.

[VAIQ_NOTE]: =>Parsing HydraSalsa...

[VAIQ_NOTE]: Start to trace and freeze model...

[VAIQ_NOTE]: The input model nndct_st_HydraSalsa_ed is torch.nn.Module.

[VAIQ_NOTE]: Finish tracing.

[VAIQ_NOTE]: Processing ops...


██████████████████████████████████████████████████| 286/286 [00:00<00:00, 911.79it/s, OpInfo: name = return_0, type = Return]                                                                                            



[VAIQ_NOTE]: =>Quantizable module is generated.(.vai_qat/HydraSalsa.py)

[VAIQ_NOTE]: =>Exporting quant config.(.vai_qat/quant_info.json)

[VAIQ_NOTE]: Start to trace and freeze model...

[VAIQ_NOTE]: The input model nndct_st_HydraSalsa_ed is torch.nn.Module.

[VAIQ_NOTE]: Finish tracing.

[VAIQ_NOTE]: Processing ops...


██████████████████████████████████████████████████| 286/286 [00:00<00:00, 1408.94it/s, OpInfo: name = return_0, type = Return]                                                                                           



[VAIQ_WARN][QUANTIZER_TORCH_OP_TYPE_MISMATCH]: Module class name and op type name do not match: Upsample vs. interpolate (Node: HydraSalsa::HydraSalsa/Salsa[salsa]/SalsaDecoder[decoder1]/UpBlock[upBlock1]/Upsample[up]/ret.219)

[VAIQ_WARN][QUANTIZER_TORCH_OP_TYPE_MISMATCH]: Module class name and op type name do not match: Upsample vs. interpolate (Node: HydraSalsa::HydraSalsa/Salsa[salsa]/SalsaDecoder[decoder1]/UpBlock[upBlock2]/Upsample[up]/ret.255)

[VAIQ_WARN][QUANTIZER_TORCH_OP_TYPE_MISMATCH]: Module class name and op type name do not match: Upsample vs. interpolate (Node: HydraSalsa::HydraSalsa/Salsa[salsa]/SalsaDecoder[decoder1]/UpBlock[upBlock3]/Upsample[up]/ret.291)

[VAIQ_WARN][QUANTIZER_TORCH_OP_TYPE_MISMATCH]: Module class name and op type name do not match: Upsample vs. interpolate (Node: HydraSalsa::HydraSalsa/Salsa[salsa]/SalsaDecoder[decoder1]/UpBlock[upBlock4]/Upsample[up]/ret.327)

[VAIQ_WARN][QUANTIZER_TORCH_OP_TYPE_MISMATCH]: Module class name and op type name do no

In [26]:
torch.cuda.empty_cache()

In [ ]:
from pytorch_nndct import quantization as quant

# Load your model
model = torch.load('your_model.pth')
model.eval()

# Prepare your data loader
from torch.utils.data import DataLoader
# Replace with your dataset and preprocessing
data_loader = DataLoader(your_dataset, batch_size=1, shuffle=False)

# Quantize your model
quant_model = quant.Quantizer(model, output_dir='quant_results', device=torch.device('cuda'))

# Calibrate with a few batches
for data, target in data_loader:
    quant_model(data)

# Save the quantized model
quant_model.save_model('quantized_model.pth')
